# Dataset Transforms

Based on **Patric Loeber** video: https://www.youtube.com/watch?v=c36lUUr864M&t=8008s

import torch
import torchvision

# we are using built-in dataset
# we can pass the transform argument to this dataset and then apply some transforms
# in this example we use the MNIST dataset and then we apply to tensor transform
# which will convert images or numpy arrays to tensors
dataset = torchvision.datasets.MNIST(download=True,
    root='./data', transform=torchvision.transforms.ToTensor())

In [4]:
print(dataset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()


## Transforms

Transforms can be applied to PIL images, tensors, ndarrays or custom data during creation of the DataSet

complete list of built-in transforms:
https://pytorch.org/vision/0.9/transforms.html

### On Images

CenterCrop, Grayscale, Pad, RandomAffine, RandomCrop, RandomHorizontalFlip, RandomRotation, Resize, Scale

### On Tensors

LinearTransformation, Normalize, RandomErasing

### Conversion

**ToPILImage**: from tensor or ndarray <br>
**ToTensor**: from numpy.ndarray or PILImage

### Generic

Use Lambda

### Custom

Write own class

### Compose multiple Transforms

```
composed = transforms.Compose([Rescale(256),
                                RandomCrop(224)])
                                
torchvision.transforms.Rescale(256)
torchvision.transforms.ToTensor()
```

## Extending WineDataset to support transforms

In [9]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np

class WineDataset(Dataset):
    
    def __init__(self, transform=None):
        # data loading
        xy = np.loadtxt('./data/wine_data/wine.csv', delimiter=",", dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]
        
        # we do not convert to tensor here
        self.x = xy[:, 1:]
        self.y = xy[:, [0]] # n_samples, 1
        
        self.transform = transform
        
    def __getitem__(self, index):
        # dataset[0]
        sample = self.x[index], self.y[index]
        
        # if transform != None then transform
        if self.transform:
            sample = self.transform(sample)
        
        return sample
        
    def __len__(self):
        # len(dataset)
        return self.n_samples

## Custom transform classes

In [7]:
class ToTensor():
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

In [11]:
dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]
features, labels = first_data
print(f"type of features: {type(features)}, type of labels: {type(labels)}")

type of features: <class 'torch.Tensor'>, type of labels: <class 'torch.Tensor'>


In [17]:
dataset = WineDataset(transform=None)
first_data = dataset[0]
features, labels = first_data
print(f"type of features: {type(features)}, type of labels: {type(labels)}")
print(f"features: {features}")

type of features: <class 'numpy.ndarray'>, type of labels: <class 'numpy.ndarray'>
features: [1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]


In [15]:
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, sample):
        inputs, target = sample
        inputs *= self.factor
        return inputs, target

## Composed transformations

In [18]:
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
print(f"type of features: {type(features)}, type of labels: {type(labels)}")
print(f"features: {features}")

type of features: <class 'torch.Tensor'>, type of labels: <class 'torch.Tensor'>
features: tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03])


In [19]:
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(4)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
print(f"type of features: {type(features)}, type of labels: {type(labels)}")
print(f"features: {features}")

type of features: <class 'torch.Tensor'>, type of labels: <class 'torch.Tensor'>
features: tensor([5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
        1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
        4.2600e+03])
